This code will create database of all bridges

In [30]:
import pandas as pd

# Define the years for which you have data
years = range(2012, 2024)  # This assumes you have data from 2012 to 2023

# Create an empty dictionary to store DataFrames
dataframes = {}

# Loop through the years and read the corresponding CSV files
for year in years:
    file_path = f'Data/processed_data_{year}.csv'  # Adjust the file name pattern as needed
    df = pd.read_csv(file_path, delimiter=',')
    dataframes[f'df_x{year}'] = df

# Now, you have a dictionary with DataFrames for each year, e.g., dataframes['df_x2012'] for 2012 data.


In [32]:
import pandas as pd

# Create an empty dataframe to store the merged results
merged_df = pd.DataFrame()

# Loop through the dataframes
for year, df in dataframes.items():
    if 'ID mosta' in df.columns and 'Stavebno-technický stav' in df.columns:
        # Select the columns of interest from the current dataframe
        if year == "df_x2023":
            # df_subset = df[['ID mosta', 'Stavebno-technický stav',
            #                 "Okres", "Latitude", "Longitude", "Material",
            #                 'Názov mosta', 'Dĺžka premostenia / m',
            #                 'Rok postavenia', 'Volná šírka mosta/ m',
            #                 'Normána zaťažiteľnost / t', 'Trieda PK',
            #                 'Druh konštrukcie', "Predmet premostenia",
            #                 "Správca úseku", "Číslo PK"]]
            df_subset = df[["Trieda PK", "Číslo PK", "Most /DC", "Názov mosta", "ID mosta", "ID DC",
                            "správcovské číslo", "Stavebno-technický stav", "Longitude", "Latitude",
                            "Vyska", "Druh konštrukcie", "Material", "Normána zaťažiteľnost / t",
                            "Výhradná zaťažiteľnosť / t", "Výnimočná zaťažiteľnosť / t",
                            "Spôsob stanovenia", "Predmet premostenia", "Rok postavenia",
                            "Zaťažovacia trieda STN", "Počet otvorov", "Dĺžka premostenia / m",
                            "Dĺžka nosnej konštrukcie / m", "Volná šírka mosta/ m",
                            "Šírka medzi obrubami/ m", "Pocha mosnej konštrukcie / m2",
                            "Plocha mosta / m2", "Vlastník úseku", "Správca úseku", "Okres",
                            "Smer úseku", "Typ úseku", "Počiatočný uzol", "Koncový uzol",
                            "úsek ID", "úsekové staničenie začiatku DC / m",
                            "Kilometrovníkové staničenie začiatku DC / km",
                            "Kumulatívne staničenie začiatku DC / km", "Poradie"]]
        else:
            df_subset = df[['ID mosta', 'Stavebno-technický stav']]
        # Merge the current dataframe with the existing merged_df
        if merged_df.empty:
            # If merged_df is empty, set it to the current dataframe
            merged_df = df_subset.copy()
        else:
            # Merge the current dataframe with merged_df
            merged_df = pd.merge(merged_df, df_subset, on='ID mosta', how='outer', suffixes=('', f'_{year}'))
    else:
        print(f'Columns not found in {year} dataframe.')



# Rename the "Stavebno-technický stav" column
merged_df.rename(columns={'Stavebno-technický stav': 'Stavebno-technický stav_df_x2012'}, inplace=True)


# Create a dictionary to map the old column names to the new ones
column_mapping = {}
for year in range(2012, 2024):
    old_column_name = f'Stavebno-technický stav_df_x{year}'
    new_column_name = str(year)
    column_mapping[old_column_name] = new_column_name

# Rename the columns using the dictionary
merged_df.rename(columns=column_mapping, inplace=True)

#### REGIONS
# Define the region-to-district mapping
region_mapping = {
    'Bratislava': ['Bratislava I', 'Bratislava II', 'Bratislava III', 'Bratislava IV', 'Bratislava V', 'Malacky', 'Pezinok', 'Senec'],
    'Trnava': ['Dunajská Streda', 'Galanta', 'Hlohovec', 'Piešťany', 'Senica', 'Skalica', 'Trnava'],
    'Trenčín': ['Bánovce nad Bebravou', 'Ilava', 'Myjava', 'Nové Mesto nad Váhom', 'Partizánske', 'Považská Bystrica', 'Prievidza', 'Púchov', 'Trenčín'],
    'Nitra': ['Komárno', 'Levice', 'Nitra', 'Nové Zámky', 'Šaľa', 'Topoľčany', 'Zlaté Moravce'],
    'Žilina': ["Turčianske Teplice", 'Bytča', 'Dolný Kubín', 'Kysucké Nové Mesto', 'Liptovský Mikuláš', 'Martin', 'Námestovo', 'Ružomberok', 'Tvrdošín', "Čadca", "Žilina"],
    'Banská Bystrica': ['Banská Štiavnica', "Rimavská Sobota", "Žiar nad Hronom", "Žarnovica", 'Banská Bystrica', 'Brezno', 'Detva', 'Krupina', 'Lučenec', 'Poltár', 'Revúca', 'Veľký Krtíš', 'Zvolen'],
    'Prešov': ["Svidník", "Stará Ľubovňa", 'Bardejov', 'Humenné', 'Kežmarok', 'Levoča', 'Medzilaborce', 'Poprad', 'Prešov', 'Sabinov', 'Snina', 'Spišská Nová Ves', 'Stropkov', 'Vranov nad Topľou'],
    'Košice': ['Gelnica', 'Košice I', 'Košice II', 'Košice III', 'Košice IV', 'Košice - okolie', 'Michalovce', 'Rožňava', 'Sobrance', 'Trebišov']
}

# Create a function to map districts to regions
def map_to_region(district):
    for region, districts in region_mapping.items():
        if district in districts:
            return region
    return 'Nezistený'  # If not found, return 'Unknown'

# Apply the mapping function to create the 'Region' column
merged_df['Kraj'] = merged_df['Okres'].apply(map_to_region)

# Display the filtered dataframe
merged_df = merged_df.drop_duplicates(subset="ID mosta")

merged_df.fillna("Nezistený", inplace=True)

# Define a mapping of conditions to their respective values
condition_mapping = {
    'Zlý': 5,
    'Veľmi zlý': 6,
    'Uspokojivý': 4,
    'Bezchybný': 1,
    'Dobrý': 3,
    'Havarijný': 7,
    'Nezistený': 0,
    'Veľmi dobrý': 2
}

# Loop through each year and create a new column
for year in years:
    condition_col_name = f'n_{year}'
    merged_df[condition_col_name] = merged_df[str(year)].replace(condition_mapping)

# Now, merged_df has new columns n_2012, n_2013, ..., n_2023 with the specified values for each condition.

merged_df.reset_index(drop=True, inplace=True)

merged_df.rename(columns={
    'Dĺžka premostenia / m': 'Dĺžka premostenia',
    'Normána zaťažiteľnost / t': 'Normána zaťažiteľnost',
    'Volná šírka mosta/ m': 'Volná šírka mosta',
}, inplace=True)

# Print the columns of the merged dataframe
print(merged_df.columns, len(merged_df.columns))

Index(['ID mosta', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', 'Trieda PK', 'Číslo PK', 'Most /DC',
       'Názov mosta', 'ID DC', 'správcovské číslo', '2023', 'Longitude',
       'Latitude', 'Vyska', 'Druh konštrukcie', 'Material',
       'Normána zaťažiteľnost', 'Výhradná zaťažiteľnosť / t',
       'Výnimočná zaťažiteľnosť / t', 'Spôsob stanovenia',
       'Predmet premostenia', 'Rok postavenia', 'Zaťažovacia trieda STN',
       'Počet otvorov', 'Dĺžka premostenia', 'Dĺžka nosnej konštrukcie / m',
       'Volná šírka mosta', 'Šírka medzi obrubami/ m',
       'Pocha mosnej konštrukcie / m2', 'Plocha mosta / m2', 'Vlastník úseku',
       'Správca úseku', 'Okres', 'Smer úseku', 'Typ úseku', 'Počiatočný uzol',
       'Koncový uzol', 'úsek ID', 'úsekové staničenie začiatku DC / m',
       'Kilometrovníkové staničenie začiatku DC / km',
       'Kumulatívne staničenie začiatku DC / km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014'

C:\Users\Doktor\AppData\Local\Temp\ipykernel_17712\1058089823.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Nezistený' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_df.fillna("Nezistený", inplace=True)


In [33]:
# Define a mapping for column name replacements
column_mapping = {
    'ID mosta': 'ID_mosta',
    'Trieda PK': 'Trieda_PK',
    'Číslo PK': 'Číslo_PK',
    'Most /DC': 'Most_DC',
    'Názov mosta': 'Názov_mosta',
    'ID DC': 'ID_DC',
    'správcovské číslo': 'správcovské_číslo',
    'Longitude': 'Zemepisná_dĺžka',
    'Latitude': 'Zemepisná_šírka',
    'Vyska': 'Výška',
    'Normána zaťažiteľnost': 'Normálna_zaťažiteľnosť',
    'Výhradná zaťažiteľnosť / t': 'Výhradná_zaťažiteľnosť_t',
    'Výnimočná zaťažiteľnosť / t': 'Výnimočná_zaťažiteľnosť_t',
    'Spôsob stanovenia': 'Spôsob_stanovenia',
    'Predmet premostenia': 'Predmet_premostenia',
    'Rok postavenia': 'Rok_postavenia',
    'Zaťažovacia trieda STN': 'Zaťažovacia_trieda_STN',
    'Počet otvorov': 'Počet_otvorov',
    'Dĺžka premostenia': 'Dĺžka_premostenia',
    'Dĺžka nosnej konštrukcie / m': 'Dĺžka_nosnej_konštrukcie_m',
    'Volná šírka mosta': 'Volná_šírka_mosta',
    'Šírka medzi obrubami/ m': 'Šírka_medzi_obrubami_m',
    'Pocha mosnej konštrukcie / m2': 'Plocha_mosnej_konštrukcie_m2',
    'Plocha mosta / m2': 'Plocha_mosta_m2',
    'Vlastník úseku': 'Vlastník_úseku',
    'Správca úseku': 'Správca_úseku',
    'Okres': 'Okres',
    'Smer úseku': 'Smer_úseku',
    'Typ úseku': 'Typ_úseku',
    'Počiatočný uzol': 'Počiatočný_uzol',
    'Koncový uzol': 'Koncový_uzol',
    'úsek ID': 'úsek_ID',
    'úsekové staničenie začiatku DC / m': 'úsekové_staničenie_začiatku_DC_m',
    'Kilometrovníkové staničenie začiatku DC / km': 'Kilometrovníkové_staničenie_začiatku_DC_km',
    'Kumulatívne staničenie začiatku DC / km': 'Kumulatívne_staničenie_začiatku_DC_km',
    'Druh konštrukcie': 'Druh_konštrukcie'
}

# Rename the columns using the mapping
merged_df.rename(columns=column_mapping, inplace=True)



columns_to_remove = [str(i) for i in range(2012, 2024)]



# Remove the specified columns


merged_df = merged_df.drop(columns=columns_to_remove)



# Print the columns of the merged dataframe


merged_df.columns, len(merged_df.columns)

(Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
        'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
        'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
        'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
        'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
        'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
        'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
        'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
        'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
        'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
        'úsekové_staničenie_začiatku_DC_m',
        'Kilometrovníkové_staničenie_začiatku_DC_km',
        'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
        'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
        'n_2020', 'n_2021', 'n_2022', 'n

In [34]:
# Replace non-integer values with NaN
merged_df['Rok_postavenia'] = pd.to_numeric(
    merged_df['Rok_postavenia'], errors='coerce')
merged_df['úsek_ID'] = pd.to_numeric(merged_df['úsek_ID'], errors='coerce')

# Convert to integer, and replace NaN values with a default integer (e.g., -1)
merged_df['Rok_postavenia'] = merged_df['Rok_postavenia'].fillna(
    -1).astype(int)
merged_df['úsek_ID'] = merged_df['úsek_ID'].fillna(-1).astype(int)

merged_df["Zemepisná_dĺžka"] = pd.to_numeric(
    merged_df["Zemepisná_dĺžka"], errors='coerce')
merged_df["Zemepisná_šírka"] = pd.to_numeric(
    merged_df["Zemepisná_šírka"], errors='coerce')
merged_df["Výška"] = pd.to_numeric(merged_df["Výška"], errors='coerce')

In [35]:
# Define the file path for saving the CSV
output_csv_path = "Data/Bridges_processed.csv"

# Save the DataFrame to a CSV file
merged_df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [36]:
merged_df.iloc[0]

ID_mosta                                                                               M5897
Trieda_PK                                                                    cesta I. triedy
Číslo_PK                                                                                   2
Most_DC                                                                      Dilatačný celok
Názov_mosta                                   02_002 Most cez potok Brestovec v obci Kopčany
ID_DC                                                                               M5897.01
správcovské_číslo                                                                          2
Zemepisná_dĺžka                                                                    17.123319
Zemepisná_šírka                                                                    48.790152
Výška                                                                              163.81625
Druh_konštrukcie                                                      